<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Denmark_Copenhagen_Meteorological_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
base_url = "https://opendataapi.dmi.dk/v2/metObs/collections/observation/items"
start_date = "2024-01-01T00:00:00Z"
end_date = "2025-01-01T00:00:00Z"
limit = 300000

tasks = [
    # Station 06180 (Airport) for Wind/Temp (10-min resolution -> converted to Hourly)
    {"station": "06180", "param": "temp_dry",   "name": "Temperature_C"},
    {"station": "06180", "param": "wind_speed", "name": "Wind_Speed_ms"},

    # Station 06187 (Toldbod) for Radiation
    {"station": "06187", "param": "radia_glob_past1h", "name": "Global_Radiation_Wm2"}
]

dfs = []
print("Fetching 2024 Data (Converting 10-min raw data to Hourly)...")

for task in tasks:
    print(f"   > Requesting {task['name']} from Station {task['station']}...")

    params = {
        "stationId": task['station'],
        "datetime": f"{start_date}/{end_date}",
        "parameterId": task['param'],
        "limit": limit
    }

    try:
        r = requests.get(base_url, params=params)

        # Fallback for Radiation if 'past1h' is missing
        if r.status_code != 200 and task['name'] == "Global_Radiation_Wm2":
            print("     'past1h' not found, trying instant 'radia_glob'...")
            params['parameterId'] = 'radia_glob'
            r = requests.get(base_url, params=params)

        if r.status_code == 200:
            data = r.json().get('features', [])
            raw_count = len(data)

            records = []
            for item in data:
                records.append({
                    'Time': item['properties']['observed'],
                    task['name']: item['properties']['value']
                })

            df_temp = pd.DataFrame(records)
            if not df_temp.empty:
                df_temp['Time'] = pd.to_datetime(df_temp['Time'])

                # --- CRITICAL STEP: CONVERT 10-MIN TO HOURLY ---
                # We group by hour and take the MEAN (average)
                df_temp = df_temp.set_index('Time').resample('h').mean()

                hourly_count = len(df_temp)
                print(f"     Success! Condensed {raw_count} raw readings into {hourly_count} hourly averages.")
                dfs.append(df_temp)
            else:
                print(f"     WARNING: Zero rows found for {task['name']}.")
        else:
            print(f"     Error {r.status_code}: {r.text}")

    except Exception as e:
        print(f"     Connection failed: {e}")
    time.sleep(1)

# --- MERGE & FORMAT ---
if dfs:
    print("Merging datasets...")
    df_final = pd.concat(dfs, axis=1)
    df_final = df_final.sort_index()

    # Fill gaps (Linear interpolation)
    df_final = df_final.interpolate(method='linear', limit=2)

    # Radiation Cleanup
    if 'Global_Radiation_Wm2' in df_final.columns:
        df_final['Global_Radiation_Wm2'] = df_final['Global_Radiation_Wm2'].fillna(0)
        df_final.loc[df_final['Global_Radiation_Wm2'] < 0, 'Global_Radiation_Wm2'] = 0

    df_final = df_final.dropna(how='all')

    # --- FORMATTING DATE & TIME ---
    # 1. Reset index so 'Time' is a column again
    df_final = df_final.reset_index()

    # 2. Create 'Date' (YYYY-MM-DD)
    df_final['Date'] = df_final['Time'].dt.strftime('%Y-%m-%d')

    # 3. Create 'Hour' (HH:MM)
    df_final['Hour'] = df_final['Time'].dt.strftime('%H:%M')

    # 4. Select and Reorder columns
    cols = ['Date', 'Hour', 'Temperature_C', 'Global_Radiation_Wm2', 'Wind_Speed_ms']
    cols = [c for c in cols if c in df_final.columns] # Safety check
    df_final = df_final[cols]

    print("-" * 30)
    print(df_final.head())
    print("-" * 30)

    filename = "Copenhagen_Formatted_2024.csv"
    df_final.to_csv(filename, index=False)
    print(f"Success! File saved as: {filename}")
else:
    print("Failed to retrieve data.")

Fetching 2024 Data (Converting 10-min raw data to Hourly)...
   > Requesting Temperature_C from Station 06180...
     Success! Condensed 52671 raw readings into 8784 hourly averages.
   > Requesting Wind_Speed_ms from Station 06180...
     Success! Condensed 52671 raw readings into 8784 hourly averages.
   > Requesting Global_Radiation_Wm2 from Station 06187...
     Success! Condensed 8777 raw readings into 8785 hourly averages.
Merging datasets...
------------------------------
         Date   Hour  Temperature_C  Global_Radiation_Wm2  Wind_Speed_ms
0  2024-01-01  00:00       5.433333                   0.0       5.233333
1  2024-01-01  01:00       5.466667                   0.0       4.183333
2  2024-01-01  02:00       5.483333                   0.0       5.383333
3  2024-01-01  03:00       5.450000                   0.0       5.400000
4  2024-01-01  04:00       5.650000                   0.0       4.600000
------------------------------
Success! File saved as: Copenhagen_Formatted_20